In [ ]:
import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm # Use the notebook-friendly version of tqdm
import ipywidgets as widgets
from IPython.display import display, clear_output

class PredatorPreyChemostat:
    """
    A class to encapsulate the predator-prey model in a chemostat,
    based on the paper "Crossing the Hopf Bifurcation in a Live Predator-Prey System."
    """
    def __init__(self, bc=3.3, Kc=4.3, bB=2.25, KB=15, m=0.055, lam=0.4, eps=0.25, delta=0.6, Ni=80.0):
        """
        Initializes the model with its parameters.
        Default values are taken from note 18 in the paper.
        """
        self.params = {
            'bc': bc, 'Kc': Kc, 'bB': bB, 'KB': KB,
            'm': m, 'lam': lam, 'eps': eps, 'delta': delta, 'Ni': Ni
        }

    def _model_equations(self, t, y):
        """
        Defines the system of four differential equations.
        Enforces non-negative populations for solver stability.
        """
        N, C, R, B = y
        C = max(0, C)
        R = max(0, R)
        B = max(0, B)
        p = self.params
        
        # Add a small epsilon to denominators to prevent division by zero
        Fc = p['bc'] * N / (p['Kc'] + N + 1e-9)
        Fb = p['bB'] * C / (p['KB'] + C + 1e-9)
        
        dN_dt = p['delta'] * (p['Ni'] - N) - Fc * C
        dC_dt = Fc * C - (Fb * B / p['eps']) - p['delta'] * C
        dR_dt = Fb * R - (p['delta'] + p['m'] + p['lam']) * R
        dB_dt = Fb * R - (p['delta'] + p['m']) * B
        
        return [dN_dt, dC_dt, dR_dt, dB_dt]

    def run_simulation(self, y0, t_span, t_eval, rtol=1e-6, atol=1e-9):
        """
        Runs the simulation using SciPy's solve_ivp with the LSODA solver,
        which is robust for potentially stiff systems.
        """
        sol = solve_ivp(
            self._model_equations, t_span, y0,
            t_eval=t_eval, method='LSODA',
            rtol=rtol, atol=atol
        )
        
        if sol.status != 0:
            return {} # Return an empty dict on failure
            
        return {
            'time': sol.t, 'N': sol.y[0], 'Chlorella': sol.y[1],
            'Reproducing_Brachionus': sol.y[2], 'Total_Brachionus': sol.y[3]
        }

In [ ]:
# 1. Define the widgets for the parameters we want to control.
delta_slider = widgets.FloatSlider(
    value=0.95,  # A value in the oscillatory regime
    min=0.1,
    max=1.5,
    step=0.01,
    description='Dilution Rate (δ):',
    continuous_update=False, # Important: only updates when we release the slider
    readout_format='.2f'
)

Ni_slider = widgets.FloatSlider(
    value=80.0,
    min=20.0,
    max=500.0,
    step=10.0,
    description='Nitrogen Inflow (Ni):',
    continuous_update=False,
    readout_format='.1f'
)

# 2. Creating an Output widget to hold and display the plots.
output_plot = widgets.Output()

# 3. Defining the function that runs the simulation and creates the plot.
# This function will be called whenever a slider's value changes.
def update_simulation_plot(delta, Ni):
    """Runs simulation and updates the plot within the Output widget."""
    with output_plot:
        # Clears the previous plot first
        clear_output(wait=True)
        
        # Sets up and runs the simulation with the new parameters
        system = PredatorPreyChemostat(delta=delta, Ni=Ni)
        initial_conditions = [60.0, 10.0, 5.0, 5.0]
        simulation_time = (0, 300) # Increased time to see long-term behavior
        t_eval_points = np.linspace(simulation_time[0], simulation_time[1], 1500)
        
        results = system.run_simulation(initial_conditions, simulation_time, t_eval_points)
        
        # Creating the figure and axes for plotting
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
        
        # Plot 1: Time Series
        ax1.set_title('Population Dynamics Over Time')
        ax1.set_xlabel('Time (days)')
        ax1.set_ylabel('Prey (Chlorella)', color='g')
        ax1.plot(results['time'], results['Chlorella'], color='g')
        ax1.tick_params(axis='y', labelcolor='g')
        ax1.grid(True)
        
        ax1_twin = ax1.twinx()
        ax1_twin.set_ylabel('Predator (Brachionus)', color='k')
        ax1_twin.plot(results['time'], results['Total_Brachionus'], color='k')
        ax1_twin.tick_params(axis='y', labelcolor='k')
        
        # Plot 2: Phase Portrait
        ax2.set_title('Phase Space Portrait')
        ax2.set_xlabel('Prey Population (Chlorella)')
        ax2.set_ylabel('Predator Population (Brachionus)')
        ax2.plot(results['Chlorella'], results['Total_Brachionus'], color='darkblue', lw=1)
        ax2.grid(True)
        
        fig.tight_layout()
        plt.show()

# 4. Link the sliders to the update function.
widgets.interactive_output(
    update_simulation_plot, 
    {'delta': delta_slider, 'Ni': Ni_slider}
)

# 5. Display the user interface.
controls = widgets.VBox([delta_slider, Ni_slider])
dashboard = widgets.VBox([controls, output_plot])

# Finally, display the complete dashboard
display(dashboard)